**Importing useful headers**


---



In [ ]:
!pip install pycrypto
from decimal import *
from random import randint  
from math import ceil, log
from base64 import b64decode
from Crypto.PublicKey import RSA
from Crypto.Util.number import getPrime
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify

**Implementing generic Functions that will be used in other questions**


---



In [ ]:
'''                         Modulo exponential using SQUARE AND MULTIPLY returns a^e mod n '''
def modulo_exponent(a,e,n):
  result=1
  a= a%n
  while e>0:
    # If odd then multiply and update the result
    if e%2!=0:
      result=(result*a)%n
     # If even then
    e=e//2
    a=(a**2)%n
  return result
#print(modulo_exponent(11,12,10))

'''                         Miller Rabin Test ALGORITHM to Test Prime Numbers return true if n is probably prime otherwise false'''
def miller_robin_Test(d,n):
  # Random number 
  a = randint(1,n-4)+2
  # Compute a^e % n
  x = modulo_exponent(a, d, n)
  if (x == 1 or x == n - 1):
    return True
    # Finding sqaure modulo till any of the following criteria doesnt match
    # (x^2) % n is not 1
    # (x^2) % n is not n-1
    # d does not reach n-1
  while (d != n - 1):
      x = (x * x) % n
      d *= 2;
      if (x == 1):
          return False
      if (x == n - 1):
          return True
 
    # Return most probably composite
  return False;

'''                       This function test prime for atleast 10 number of iteration'''
def check_prime_with_millerrobin(number,iteration=10):
  if number==2 or number==3:
    return True
  if number<=4:
    return False
  # Here we need to find a 'e', such that 2 ^ k * e +1  where k = number-1 
  # Note k is always even incase of prime
  k = number-1
  while k % 2 ==0:
    k = k//2
  for i in range(iteration):
    if not(miller_robin_Test(k,number)):
      return False
  return True
# Testing Miller robin test for 10 digit prime
# print(check_prime_with_millerrobin(5915587277))


'''                     GCD of two numbers based on Euclidean algorithm.'''
def gcd(a,b):
  if a==0:
    return b
  return gcd(b%a,a)
# Testing gcd
# print(gcd(10,35))

'''                     GCD of two numbers based on Extended Euclidean algorithm.in ax+by=gcd(a,b) it returns x,y and gcd'''
def gcd_extended(a,b):
  if a==0:
    return b,0,1   # assumed x will be also zero
  gcd_recursion, x_recursion, y_recursion=gcd_extended(b%a,a)
  x=y_recursion-(b//a)*x_recursion
  y=x_recursion
  return gcd_recursion,x,y

# Testing extended_gcd
# print(gcd_extended(10,35))

'''                       Converts the given int n to bytes.'''
def int_to_bytes(n):
    parameter=(n.bit_length() + 7) // 8
    return n.to_bytes(parameter, 'big')

# Testing int_to_bytes
# print(1024)

'''                     Computing LCM with the help of gcd formula is a*b/gcd(a,b)'''
def LCM(a,b):
  gcd_val=gcd(a,b)
  return a//gcd_val*b

# Testing LCM
# LCM(2,3)


'''                           Multiplicative Inverse using extended Euclidean'''
def modulo_inverse(a,n):
  # Used Extended Euclidean Algorithm to get x and y
  gcd_val,x,y=gcd_extended(a,n)
  if gcd_val!=1:
    raise Exception("Inverse Modulo does'nt exist!")
  else:
    mod_inverse=(x%n+n)%n
    return mod_inverse

# Testing Modulo_Inverse ans should be 2753
# print(modulo_inverse(17,3120))



**Question Set 5 challenge 39 Implement RSA**          


---


Generate 2 random primes. We'll use small numbers to start, so you can just pick them out of a prime table. Call them "p" and "q".
Let n be p * q. Your RSA math is modulo n.
Let et be (p-1)*(q-1) (the "totient"). You need this value only for keygen.
Let e be 3.
Compute d = invmod(e, et). invmod(17, 3120) is 2753.
Your public key is [e, n]. Your private key is [d, n].
To encrypt: c = m**e%n. To decrypt: m = c**d%n
Test this out with a number, like "42".
Repeat with bignum primes (keep e=3).

In [ ]:
'''Implementing RSA Encryption and Decryption Class. Note: This will be used in further question'''
class RSA:
    def __init__(self, keySIZE):
      et=0
      # Let e be 3.
      self.e=3
      # Key length will be used to compute  p and q
      while gcd(self.e, et) != 1:
        # Generate 2 random primes. We'll use small numbers to start, so you can just pick them out of a prime table. Call them "p" and "q".
          p = getPrime(keySIZE)
          q = getPrime(keySIZE)
          if check_prime_with_millerrobin(p) and check_prime_with_millerrobin(q):
            # Let et be (p-1)*(q-1) (the "totient"). You need this value only for keygen.
            et = LCM(p - 1, q - 1)
            # Let n be p * q.
            self.n = p * q 
      # Compute d = invmod(e, et). invmod(17, 3120) is 2753.   ALL PARAMETERS USED AS PER QUESTION LIKE Extended Euclidean Algorithm
      self.d = modulo_inverse(self.e, et)
    '''                                                 To encrypt: c = m**e%n'''
    def encrypt(self, plaintext):
      # To encrypt: c = m**e%n
      m = int.from_bytes(plaintext, byteorder='big')
      return modulo_exponent(m, self.e, self.n)

    '''                                                 To decrypt: m = c**d%n '''
    def decrypt(self, c):
      # To decrypt: m = c**d%n
      m = modulo_exponent(c, self.d, self.n)
      return int_to_bytes(m)
   

In [ ]:
# Diver for Question 39:
if __name__ == '__main__':
  # Testing RSA
  key_length = 460
  # Typically working for key_length >= 500
  message = b"HOMEWORK 4 QUESTION 39 SET 5 IMPLEMENTED BY SHRISTY GUPTA"
  rsa_object = RSA(key_length)
  cipher_text = rsa_object.encrypt(message)
  plain_text = rsa_object.decrypt(cipher_text)
  print("Cipher Text: ",cipher_text)
  print("Plain Text: ", plain_text)

Cipher Text:  2316498238096631462378403190159469420642947611070931719967975345458006156219407200999150532060679251768142583825503223529366000219654325035292184570852500337810648426180303652689476735611551288857273908252934636475658229984046307798059908058250416689284455661016243113818434362
Plain Text:  b'HOMEWORK 4 QUESTION 39 SET 5 IMPLEMENTED BY SHRISTY GUPTA'


**Challenge 40 Set 5 Implement an E=3 RSA Broadcast attack**   


---



> Assume you're a Javascript programmer. That is, you're using a naive handrolled RSA to encrypt without padding.


> Assume you can be coerced into encrypting the same plaintext three times, under three different public keys. You can; it's happened.



> Then an attacker can trivially decrypt your message, by:



> Capturing any 3 of the ciphertexts and their corresponding pubkeys
Using the CRT to solve for the number represented by the three ciphertexts (which are residues mod their respective pubkeys)
Taking the cube root of the resulting number



In [ ]:
# This code is uses concepts of chinese remainder theorem
''' Cube root found with the help of Binary search'''
def cubeRoot(n):
  start = 0
  end = n
  while start < end:
    mid = (start + end)//2
    cube = mid*mid*mid
    if cube < n:
      start = mid + 1
    else:
      end = mid
  return start
# Test cuberoot
# print(cubeRoot(27))

def RSA_Broadcast_attack(three_RSA_Ciphertext):
  #1. Capturing any 3 of the ciphertexts and their corresponding pubkeys
  c_0, n_0=three_RSA_Ciphertext[0][0],three_RSA_Ciphertext[0][1]
  c_1, n_1=three_RSA_Ciphertext[1][0],three_RSA_Ciphertext[1][1]
  c_2, n_2=three_RSA_Ciphertext[2][0],three_RSA_Ciphertext[2][1]
  # This for a1 we need to multiply [a0...an] ~ a1
  m_s_0, m_s_1, m_s_2 = n_1* n_2, n_0*n_2, n_0*n_1
  # 2.Using the CRT to solve for the number represented by the three ciphertexts (which are residues mod their respective pubkeys)
  component1 = c_0 * m_s_0 * modulo_inverse(m_s_0, n_0)
  component2 = c_1 * m_s_1 * modulo_inverse(m_s_1, n_1)
  component3 = c_2 * m_s_2 * modulo_inverse(m_s_2, n_2)
  # Given N_012 is the product of all three moduli
  N_012= n_0 * n_1 * n_2 
  result = component1 + component2 + component3
  result = result % N_012
  # 3.Taking the cube root of the resulting number
  finalresult=cubeRoot(result)
  return int_to_bytes(finalresult)


In [ ]:
# Diver for Question 40:
if __name__ == '__main__':
  # Testing RSA
  key_length = 460
  # Typically working for key_length >= 500
  message = b"you're a Javascript programmer"
  # Creating list of cipher text and n
  three_RSA_Ciphertext=[]
  step=1
  while step<=3:
        rsa_object = RSA(key_length)
        three_RSA_Ciphertext.append((rsa_object.encrypt(message), rsa_object.n))
        step=step+1
  attacked_plain_text=RSA_Broadcast_attack(three_RSA_Ciphertext)
  print("Decrypted text:",attacked_plain_text)

Decrypted text: b"you're a Javascript programmer"


**Challenge 41 Set 6 Implement unpadded message recovery oracle**


---



> Capture the ciphertext C


> Let N and E be the public modulus and exponent respectively


Let S be a random number > 1 mod N. Doesn't matter what.

> C' = ((S**E mod N) C) mod N


> Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P


Now:
          P'


> P = -----  mod N


          S



In [ ]:
decrypted = set()
def RSAServer_decypt(rsa_object,message):
  global decypted
  if message in decrypted:
    raise Exception('Cant submit the same message twice!')
  decrypted.add(message)
  return rsa_object.decrypt(message)


def unpadded_message_recovery_oracle(rsa_object,C,e,n):
  #S be a random number > 1 mod N
    while True:
        S = randint(2, n - 1)
        if S % n > 1:
            break
  # Implementing  C' = ((S**E mod N) C) mod N
    C_ = (modulo_exponent(S, e, n) * C) % n

  # Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P
    P_ = RSAServer_decypt(rsa_object,C_)
    P_ = int.from_bytes(P_, byteorder='big')

  # P=P'/S mod N
    r = (P_ * modulo_inverse(S, n)) % n
    return int_to_bytes(r)
    


In [ ]:
# Driver for Question 41:
if __name__ == '__main__':
  # Testing RSA
  key_length = 512
  # Typically working for key_length >= 500
  message = b"Nate Lawson says we should stop calling it RSA padding and start calling it RSA armoring"
  rsa_object = RSA(key_length)
  recovered_plain_text=unpadded_message_recovery_oracle(rsa_object,rsa_object.encrypt(message),rsa_object.e,rsa_object.n)
  print(recovered_plain_text)

b'Nate Lawson says we should stop calling it RSA padding and start calling it RSA armoring'


**Challenge 46 Set 6 RSA parity oracle**



---

Generate a 1024 bit RSA key pair.

Write an oracle function that uses the private key to answer the question "is the plaintext of this message even or odd" (is the last bit of the message 0 or 1). Imagine for instance a server that accepted RSA-encrypted messages and checked the parity of their decryption to validate them, and spat out an error if they were of the wrong parity.

In [ ]:
def RSA_Parity_oracle(RSA_Object,e):
  # is the plaintext of this message even or odd
  RSA_modulo = modulo_exponent(e,RSA_Object.d,RSA_Object.n)
  if RSA_modulo % 2 == 0:
    return False
  else:
    return True

# Need to achieve this after log2(n) iterations, you have the decryption of the message

def RSA_Parity_attack(RSA_Object,ciphertext):
  # Given: If you double a ciphertext (multiply it by (2**e)%n), the resulting plaintext will (obviously) be either even or odd.
  double_ciphertext=modulo_exponent(2,RSA_Object.e,RSA_Object.n)
  # Given: Need to achieve this after log2(n) iterations, you have the decryption of the message
  iteration= log(RSA_Object.n,2)
  # Taking integer upper bound
  iteration = int(ceil(iteration))
  # Taking precision of decimal numbers
  getcontext().prec = iteration
  # Given: Your decryption function starts with bounds for the plaintext of [0,n]
  start= Decimal(0)
  end= Decimal(RSA_Object.n)
  
  # Given : If the plaintext after doubling is even, doubling the plaintext didn't wrap the modulus --- the modulus is a prime number. That means the plaintext is less than half the modulus.
  while iteration>0:
    iteration=iteration-1
    # Like modulo exponent 
    ciphertext = (ciphertext * double_ciphertext)%RSA_Object.n
    # Given: Each iteration of the decryption cuts the bounds in half; either the upper bound is reduced by half, or the lower bound is.
    middle = (start + end)/2
    if RSA_Parity_oracle(RSA_Object,ciphertext) == True:
    # Given: If the plaintext after doubling is even, doubling the plaintext didn't wrap the modulus
      start = middle
    else:
    # Given: the modulus is a prime number. That means the plaintext is less than half the modulus.
      end = middle
    # Given: Print the upper bound of the message as a string at each iteration; you'll see the message decrypt "hollywood style".
    print(int_to_bytes(int(end)))
 
  return int_to_bytes(int(end))


In [ ]:
# Driver for Question 46:
if __name__ == '__main__':
  # Testing RSA Oracle
  key_length = 1024
  # Typically working for key_length >= 500
  message = b64decode("VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==")
  rsa_object = RSA(key_length)
  cipher_text = rsa_object.encrypt(message)
  recovered_message=RSA_Parity_attack(rsa_object,cipher_text)
  print(recovered_message)

b'5)\xb3\x1e\xf6Y*o\x86\xe3\x99\xa7\xadw\x984\x7f-L\xcc\xc5\xa7\xf3`\x13D\x84\xf9yOf\xb6\x98\xc6f\x84S\xd3\x056\xe1\xa4a\x97\xb5\xca|9E"\xa8\xfb\xb9N\xe2\xe3\xf5\xcf\xb6\x9b\xaf\xcd\x14\xd2\xb4\xc1e\xcd($g\xbdT\x8c\xc4\xea#\xd7\xa9\xd5+qG\xd2#\x80\xac\x05\xaft?v\xd5\xb8\x0e^\xc4\xf1\x03\xa2 1\x82\x8b\xc8\x1c\xe7\x01qG\r\xc9\x83\xc3c\xb7\x0c\xa7\xea\x9c[\x1c\x8b\xc5\xd7\x87\xcd\xe8\xe5\x91\xd1\xe3\xea\xbb-X&\xcc\xca\x17b\xc5\x14\xf5\x95\xfc5\xa8\xbe\xce\xc8\xe9\xe5N\xab\xa5V\x92%\xf7\xda_dj\xa0d\xe1\x13\x87x[\xac\x0c\x92\xaf\xbb\xe4_\x17\xc4\xf3\xdd\xecgg[\x18\xa9Y8z\x84\xa3h6lRM\xc5\xe1\x12Y1\x11\x1f\xee\xf8 ]U\x96\x1d\xf2\xb3\x87\xf9\xe3@\xdeNu\xd9b\xdf\xe2u\x9b\xac\xcby^\xd4\x8b\xea\xe2Q\xdf\xe1&9\x9c\xabFV\x95\x08\x0bY2K\xc4~\xfe}\xff\x11'
b"\x1a\x94\xd9\x8f{,\x957\xc3q\xcc\xd3\xd6\xbb\xcc\x1a?\x96\xa6fb\xd3\xf9\xb0\t\xa2B|\xbc\xa7\xb3[Lc3B)\xe9\x82\x9bp\xd20\xcb\xda\xe5>\x1c\xa2\x91T}\xdc\xa7qq\xfa\xe7\xdbM\xd7\xe6\x8aiZ`\xb2\xe6\x94\x123\xde\xaaFbu\x11\xeb\xd4\xea\x95\xb8\xa3\xe9\